In [1]:
%run ../utils/commonImports.py
%run ../utils/tradingImports.py
%matplotlib inline

from utils.common import *

# Load Data

In [2]:
ohlc = pd.read_csv('D:\\Dropbox\\My work\\krypl-project\\dataCleaned\\poloniex\\5min\\USDT_BTC_5min_2016-01-01_2017-12-31.tsv', sep='\t')\
    .query("date >= '2017-01-01'")\
    .sort_values('date').reset_index().drop('index', axis=1)

In [3]:
dataRoot = 'D:\\Dropbox\\My work\\krypl-project\\dataFeatures\\poloniex\\5min'
fileName = 'USDT_BTC_5min_2016-01-01_2017-12-31.tsv'
file = '{dataRoot}\\{fileName}'.format(dataRoot=dataRoot, fileName=fileName)

usdtBtc = pd.read_csv(file, sep='\t').query("date >= '2017-01-01'")\
    .sort_values('date').reset_index().drop('index', axis=1)
usdtBtc['timestamp'] = usdtBtc.date.apply(strTimeToTimestamp).astype(int)

trainRatio = 0.7
trainSize = int(usdtBtc.shape[0] * trainRatio)
usdtBtcTrain = usdtBtc.iloc[:trainSize]
usdtBtcTest = usdtBtc.iloc[trainSize:].reset_index().drop('index', axis=1)
ohlcTrain = ohlc.iloc[:trainSize]
ohlcTest = ohlc.iloc[trainSize:]

usdtBtcManagerTrain = CurrencyDataManager(usdtBtcTrain['close'], usdtBtcTrain[['close']])
usdtBtcManagerTest = CurrencyDataManager(usdtBtcTest['close'], usdtBtcTest[['close']])
wallet = {'usdt': 1000}
contractPair = ContractPair.new('usdt', 'btc')

# Explore

In [ ]:
%matplotlib notebook

In [ ]:
usdtBtcR.columns

In [ ]:
usdtBtcR.iloc[190:, :]

In [ ]:
step = 361
day = 288
i = day * step
j = i + day
usdtBtcR = usdtBtc.iloc[i:j, :]

paper_rc = {'lines.linewidth': 0.5, 'lines.markersize': 1}                  
sns.set_context("paper", rc = paper_rc) 

fig, ax = plotCandles(ohlc.iloc[i:j, :])
ax2 = ax.twinx()
sns.pointplot(usdtBtcR.timestamp.values, usdtBtcR['vol36'].values, ax=ax2)
sns.pointplot(usdtBtcR.timestamp.values, usdtBtcR['zscore_36_return'].values, ax=ax2, color='green')

set_date_axis(usdtBtcR['timestamp'], ax, fig)

# Implementation

In [4]:
from copy import deepcopy

class CustomStrategy:
    def __init__(self, exchange, dataManager, historyLen, contractPair, tradeSize, willingLoss, 
                 distanceFromMaxThreshold1, distanceFromMaxThreshold2, targetProfit):
        self.exchange = exchange
        self.walletStart = deepcopy(self.exchange.wallet)
        self.dataManager = dataManager
        self.historyLen = historyLen
        self.contractPair = contractPair
        self.opened = False
        self.tradeSize = tradeSize
        self.willingLoss = willingLoss
        self.distanceFromMaxThreshold1 = distanceFromMaxThreshold1
        self.distanceFromMaxThreshold2 = distanceFromMaxThreshold2
        self.targetProfit = targetProfit
    
    
    def isRisky(self, priceBought, actualPrice):
        return self.currentReturn(priceBought, actualPrice) < -self.willingLoss 
    
    
    def currentReturn(self, priceBought, actualPrice):
        feePart = (1 - self.exchange.fee) ** 2
        returnPart = (actualPrice * feePart) / priceBought
        return returnPart - 1
    
    
    def sellWhole(self, price):
        self.opened = False
        amount = self.exchange.balance(self.contractPair['tradeContract'])
        self.exchange.sell(self.contractPair, amount, price)
    
    
    def buy(self, price):
        amount = self.tradeSize / price
        self.exchange.buy(self.contractPair, amount, price)
        self.opened = True
        
        fee = self.exchange.fee
        breakEvenPrice = price / ((1 - fee) ** 2)
#         print('price', price, 'breakEven', breakEvenPrice)
        return breakEvenPrice
        
    
    def isTargetSatisffied(self, priceBought, price):
        return self.currentReturn(priceBought, price) >= self.targetProfit
    
    
    def currentTime(self):
        return self.dataManager.time
    
    def timeExceeded(self, timeBought):
        return (self.currentTime() - timeBought) > self.historyLen
    
    def trade(self):
        
        wasBellow = False
        maxPrice = -1
        while self.dataManager.has_tick():            
            history, price = self.dataManager.tick(self.historyLen)
            if history.shape[0] < self.historyLen:
                continue

            closeMax = history[0].max()
            distanceFromMax = (closeMax / price) - 1
            distanceFromMax2 = (maxPrice / price) - 1
#             print(history.shape[0], distanceFromMax, closeMax, price)
            if not self.opened and not wasBellow and distanceFromMax >= self.distanceFromMaxThreshold1:
                wasBellow = True
                maxPrice = closeMax
            elif not self.opened and wasBellow and distanceFromMax2 <= self.distanceFromMaxThreshold2:
#                 print(self.dataManager.time, 'openning trade: ', price, end='\t')
                breakEvenPrice = self.buy(price)
                priceBought = price
                timeBought = self.currentTime()
            elif self.opened and (self.isRisky(priceBought, price)):
#                 print(self.dataManager.time, 'stopLoss: ', price, self.currentReturn(breakEvenPrice, price) * 100)
                self.sellWhole(price)
                wasBellow = False
            elif self.opened and self.isTargetSatisffied(breakEvenPrice, price):
#                 print(self.dataManager.time, 'profit: ', price, self.currentReturn(breakEvenPrice, price) * 100)
                self.sellWhole(price)
                wasBellow = False
        
        if self.opened:
            self.sellWhole(price)

                
    def stats(self, contractName):
        return Statistics(contractName, self.walletStart[contractName]).evaluate(self.exchange.transactions)

# Learning

In [5]:
dataManager = deepcopy(usdtBtcManagerTrain)
exchange = BackTestExchange(dataManager, deepcopy(wallet), 0.0025)
strategy = CustomStrategy(exchange, dataManager, 1*288, contractPair, 100, willingLoss=0.05, 
                                   distanceFromMaxThreshold1=0.04, distanceFromMaxThreshold2=0.035, targetProfit=0.02)
strategy.trade()
strategy.stats('usdt').report()

,usdt
startAmount,"1,000.0000"
numberOfTrades,43.0000
totalProfit,-53.5407
avgProfit,-1.2451
winPercentage,53.4884
avgWinTrade,2.7066
avgLossTrade,-5.7896
profitFactor,0.5376
maxDrawdown,5.3670


In [ ]:
exchange.wallet

In [9]:
import time, sys
DAY = 288

allStats = None
print('historyLen willingLoss distanceFromMaxThreshold1 distanceFromMaxThreshold2 targetProfit')
for historyLen in [DAY, 2*DAY, 3*DAY, 7*DAY]:
    for willingLoss in [.01, 0.02, .03, .05, .1, .2, .5]:
        distanceThresholds = [.01, .02, .03, .04, .05, 0.06, .07, .08]
        for distanceFromMaxThreshold1 in distanceThresholds:
            for distanceFromMaxThreshold2 in [x for x in distanceThresholds if x <= distanceFromMaxThreshold1]:
                for targetProfit in [.01, .02, .03, .04, .05, .07, .1, .2, .3]:
                    s = "\r{} {} {} {} {}".format(historyLen, willingLoss, distanceFromMaxThreshold1, 
                                                  distanceFromMaxThreshold2, targetProfit)
                    sys.stdout.write(s)
                    sys.stdout.flush()

                    start_time = time.time()
                    dataManager = deepcopy(usdtBtcManagerTrain)
                    exchange = BackTestExchange(dataManager, deepcopy(wallet), 0.0025)
                    strategy = CustomStrategy(exchange, dataManager, historyLen, contractPair, 100, willingLoss=willingLoss, 
                                              distanceFromMaxThreshold1=distanceFromMaxThreshold1, 
                                              distanceFromMaxThreshold2=distanceFromMaxThreshold2, 
                                              targetProfit=targetProfit)
                    strategy.trade()
                    stats = strategy.stats('usdt').report().transpose().reset_index().rename(columns={'index': 'contract'})
                    stats['historyLen'] = historyLen
                    stats['willingLoss'] = willingLoss
                    stats['distanceFromMaxThreshold1'] = distanceFromMaxThreshold1
                    stats['distanceFromMaxThreshold2'] = distanceFromMaxThreshold2
                    stats['targetProfit'] = targetProfit
                    if allStats is None:
                        allStats = stats
                    else:
                        allStats = allStats.append(stats)
                    sys.stdout.write(s + "\t %.2f seconds" % (time.time() - start_time))
                    sys.stdout.flush()
            

allStats.to_csv('D:\\Dropbox\\My work\\krypl-project\\result\\customStrategy.tsv', index=False, sep='\t')

historyLen willingLoss distanceFromMaxThreshold1 distanceFromMaxThreshold2 targetProfit
2016 0.5 0.08 0.08 0.3	 0.97 seconds

# Look on transactions

In [ ]:
from matplotlib.finance import candlestick_ohlc
import matplotlib.dates as mdates

import time
import datetime as dt

def timestampToDate(timestamp):
    return dt.datetime.fromtimestamp(int(timestamp))

def strTimeToTimestamp(strTime):
    return dt.datetime.strptime(strTime, "%Y-%m-%d %H:%M:%S").timestamp()


def calculateMA(history, col, timePeriod):
        return calculateTalib('MA', {col: history[col].values}, {'timeperiod': timePeriod})


def plotCandles(df, currencyPair):
    DATA = df[['open', 'high', 'low', 'close', 'volume', 'date', 'timestamp']].copy()
    DATA = DATA.reset_index()
    DATA["dateStr"] = DATA["date"]
    DATA["date"]  = DATA["timestamp"].apply(timestampToDate).apply(mdates.date2num)

    plt.figure(figsize=(16, 8))
    ax = plt.gca()
    candlestick_ohlc(ax, DATA.values, width=.6, colorup='#53c156', colordown='#ff1717')

    ax.xaxis_date()
    
    xDates = DATA['dateStr']
    step = int(xDates.shape[0] / 20) + 1
    xDates[[i for i in range(xDates.shape[0]) if i % step != 0]] = ''
    
    plt.xticks(range(DATA.shape[0]), DATA['dateStr'], rotation=45)
    plt.title(currencyPair, fontproperties=titleFont)
    

def plotTransaction(transaction, i):
    print(transaction)
    color = 'k' if transaction['type'] == Transaction.BUY else 'm'
    print(color)
    plt.scatter(i, transaction['price'], s=200, c=color)
    

def plotMax(mmax, i):
    plt.scatter(i, mmax, s=200, c='b')

    
def indexRange(i, offsetBack, offsetForward, length):
    start = i - offsetBack if i - offsetBack > 0 else 0
    end = i + offsetForward if i + offsetForward < length else length-1
    return start, end
    
def plotTransactionWithSurroundings(prices, timePeriod, transaction):
    i = transaction['timestamp']
    start,end = indexRange(i, 40, 20, prices.shape[0])
    selected = prices.iloc[start:end, :].reset_index()
    
    plotCandles(selected, '')
    plotTransaction(transaction, 39)
    
    mmax = prices.iloc[start:i, :]['close'].max()
#     plotMax(mmax, 39)
    

In [ ]:
len(strategy.exchange.transactions)

In [ ]:
def plotTransactions(ohlc, transactions):
    ohlcMaxI = ohlc.shape[0]
    buys = transactionsToPlot(transactions, Transaction.BUY)
    sells = transactionsToPlot(transactions, Transaction.SELL)
    
    firstTransaction = int(min(min(buys[:, 0]), min(sells[:, 0])))
    firstTransaction = firstTransaction - 40 if firstTransaction - 40 > 0 else 0
    
    lastTransaction = int(max(max(buys[:, 0]), max(sells[:, 0])))
    lastTransaction = lastTransaction + 40 if lastTransaction + 40 < ohlcMaxI else ohlcMaxI
    
    fig, ax = plotCandles(ohlc.iloc[firstTransaction:lastTransaction, :])
    
    buyScatter = plt.scatter(buys[:, 0] - firstTransaction, buys[:, 1], s=200, c='g', label='buy')
    sellScatter = plt.scatter(sells[:, 0] - firstTransaction, sells[:, 1], s=200, c='m', label='sell')

    plt.legend(handles=[buyScatter, sellScatter], loc='upper left')

In [ ]:
%matplotlib notebook
plotTransactions(ohlcTrain, strategy.exchange.transactions[50:])

In [ ]:
t = 70

transaction = strategy.exchange.transactions[t]
plotTransactionWithSurroundings(ohlcTrain, strategy.historyLen, transaction)
transaction = strategy.exchange.transactions[t+1]
plotTransactionWithSurroundings(ohlcTrain, strategy.historyLen, transaction)

In [ ]:
transaction

# Evaluate Distance From Max

In [15]:
allStats = pd.read_csv('D:\\Dropbox\\My work\\krypl-project\\result\\customStrategy.tsv',sep='\t')

In [ ]:
allStats.query('totalProfit > 90 and winPercentage > 70').sort_values('numberOfTrades', ascending=False)

In [16]:
allStats.sort_values('totalProfit', ascending=False).query('willingLoss < 0.1').query('winPercentage > 50')

,contract,startAmount,numberOfTrades,totalProfit,avgProfit,winPercentage,avgWinTrade,avgLossTrade,profitFactor,maxDrawdown,historyLen,willingLoss,distanceFromMaxThreshold1,distanceFromMaxThreshold2,targetProfit
7760,usdt,"1,000.0000",32.0000,48.8773,1.5274,65.6250,4.1328,-3.4464,2.2893,1.0158,2016,0.0300,0.0800,0.0700,0.0300
7956,usdt,"1,000.0000",52.0000,46.9991,0.9038,86.5385,1.9173,-5.6117,2.1965,1.0252,2016,0.0500,0.0600,0.0600,0.0100
3272,usdt,"1,000.0000",25.0000,46.6079,1.8643,56.0000,7.9344,-5.8612,1.7229,1.7316,576,0.0500,0.0300,0.0100,0.0700
8021,usdt,"1,000.0000",40.0000,45.8253,1.1456,70.0000,3.9912,-5.4939,1.6951,2.5223,2016,0.0500,0.0700,0.0700,0.0300
8083,usdt,"1,000.0000",31.0000,44.3496,1.4306,80.6452,3.0468,-5.3035,2.3937,1.4579,2016,0.0500,0.0800,0.0700,0.0200
7958,usdt,"1,000.0000",41.0000,44.3236,1.0811,70.7317,3.8443,-5.5967,1.6600,1.9802,2016,0.0500,0.0600,0.0600,0.0300
7633,usdt,"1,000.0000",48.0000,42.9780,0.8954,68.7500,2.8853,-3.4824,1.8228,1.5635,2016,0.0300,0.0600,0.0600,0.0200
3281,usdt,"1,000.0000",28.0000,42.2616,1.5093,53.5714,7.9030,-5.8680,1.5540,1.7434,576,0.0500,0.0300,0.0200,0.0700
7634,usdt,"1,000.0000",45.0000,41.6832,0.9263,60.0000,3.8095,-3.3985,1.6814,1.8610,2016,0.0300,0.0600,0.0600,0.0300
7310,usdt,"1,000.0000",51.0000,41.5831,0.8154,50.9804,4.0100,-2.5071,1.6634,1.6345,2016,0.0200,0.0600,0.0600,0.0300


In [18]:
dataManager = deepcopy(usdtBtcManagerTest)
exchange = BackTestExchange(dataManager, deepcopy(wallet), 0.0025)
strategy = CustomStrategy(exchange, dataManager, 2016, contractPair, 100, willingLoss=0.05, 
                                   distanceFromMaxThreshold1=0.06, distanceFromMaxThreshold2=0.03, targetProfit=0.01)
strategy.trade()
strategy.stats('usdt').report()

,usdt
startAmount,"1,000.0000"
numberOfTrades,7.0000
totalProfit,5.7388
avgProfit,0.8198
winPercentage,85.7143
avgWinTrade,1.8997
avgLossTrade,-5.6597
profitFactor,2.0140
maxDrawdown,0.5660
